Imports

In [1]:
from __future__ import print_function
import os
import numpy as np
import sys
import tarfile
import xml.etree.ElementTree as ET
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from yaafelib import *


In [2]:
#print(os.listdir(os.getcwd()))
test_track = 'data/genres/hiphop/hiphop.00049.au'


In [127]:
class IsmisFeatures:
    
    def __init__(self):
        self.temporal_centroid = None      #1: Temporal Centroid
        self.spectral_centroid_avg = None  #2: Spectral Centroid average value
        self.spectral_centroid_var = None  #3: Spectral Centroid variance
        self.ase_per_band_avg = []         #4-37: Audio Spectrum Envelope (ASE) average values in 34 frequency bands
        self.ase_avg = None                #38: ASE average value (averaged for all frequency bands)
        self.ase_per_band_var = []         #39-72: ASE variance values in 34 frequency bands
        self.ase_var_avg = []              #73: averaged ASE variance parameters
        self.centroid_avg = None           #74: Audio Spectrum Centroid – average
        self.centroid_var = None           #75: Audio Spectrum Centroid – variance
        self.spread_avg = None             #76: Audio Spectrum Spread – average
        self.spread_var = None             #77: Audio Spectrum Spread – variance
        self.sfm_per_band_avg = []         #78-101: Spectral Flatness Measure (SFM) average values for 24 frequency bands
        self.sfm_avg = None                #102: SFM average value (averaged for all frequency bands)
        self.sfm_per_band_var = []         #103-126: Spectral Flatness Measure (SFM) variance values for 24 frequency bands
        self.sfm_var_avg = None            #127: averaged SFM variance parameters
        self.mfcc = []                     #128-147: 20 first mel cepstral coefficients average values
        
    def extract_yaafe_features():
        

    

In [130]:
def get_ismiss_feature_plan():
    fp = FeaturePlan(sample_rate=22050, normalize=1)
    # Features that seems to be most often used, so they are good to start with.
    #1 - Temporal Centroid [0]
    fp.addFeature('temporal_centroid: TemporalShapeStatistics')
    
    #2 - Spectral Centroid (3?)
    fp.addFeature('spectral_shape: SpectralShapeStatistics FFTLength=0  FFTWindow=Hanning  blockSize=1024  stepSize=512')
    
    # 78-101 - Spectral Flatness per band 
#     fp.addFeature('spectral_flattness_per_band: SpectralFlatnessPerBand FFTLength=0  FFTWindow=Hanning  blockSize=1024  stepSize=512')
    
    # 102 - Spectral Flatness (for all bands)
#     fp.addFeature('spectral_flatness: SpectralFlatness FFTLength=0  FFTWindow=Hanning  blockSize=1024  stepSize=512')
    
    #128-167
    #fp.addFeature('mfcc: MFCC CepsNbCoeffs=20')
    #fp.addFeature('zcr: ZCR')
    #fp.addFeature('spectral_shape: SpectralShapeStatistics')
    
#     fp.addFeature('envelope: Envelope EnDecim=200  blockSize=32768  stepSize=16384')
    #fp.addFeature('magnitude_spectrum: MagnitudeSpectrum')
    return fp

def get_features(audio_file, feature_plan):
    if os.path.exists(audio_file):
        print('Getting features from ' + audio_file)
    else:
        raise Exception('File ' + audio_file + ' not found')
    df = feature_plan.getDataFlow()
    engine = Engine()
    engine.load(df)
    afp = AudioFileProcessor()
    afp.setOutputFormat('csv', 'features', {'Precision': '8', 'Metadata': 'False'})
    afp.processFile(engine, audio_file)
    engine.flush()
    feats = engine.readAllOutputs()
    return feats

fp = get_ismiss_feature_plan()
features = get_features(test_track, fp)

print(features)
for i in features:
    print(i, features[i].shape)

Getting features from data/genres/hiphop/hiphop.00049.au
{'temporal_centroid': array([[ 125.60137169,   82.81068322,    0.28659876,  -46.47988592]]), 'spectral_shape': array([[ 211.31671796,  154.26268474,    0.24614688,  -33.25321142]])}
temporal_centroid (1, 4)
spectral_shape (1, 4)


In [126]:
ns = {'xmlns' : 'urn:mpeg:mpeg7:schema:2001',
      'mpeg7' : 'urn:mpeg:mpeg7:schema:2001',
      'xsi' : 'http://www.w3.org/2001/XMLSchema-instance',
      'xsi:schemaLocation' : 'urn:mpeg:mpeg7:schema:2001 http://mpeg7audioenc.sourceforge.net/mpeg7audioenc.xsd'} 


tree = ET.parse('desc.xml')
root = tree.getroot()

audio_spectrum_centroid = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumCentroidType']", ns)
audio_spectrum_spread = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumSpreadType']", ns)
audio_spectrum_envelope = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumEnvelopeType']", ns)
audio_spectrum_flatness = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumFlatnessType']", ns)




<Element '{urn:mpeg:mpeg7:schema:2001}AudioDescriptor' at 0x7f54d3f91ed0>
<Element '{urn:mpeg:mpeg7:schema:2001}AudioDescriptor' at 0x7f54d3f5aa10>
<Element '{urn:mpeg:mpeg7:schema:2001}AudioDescriptor' at 0x7f54d3f84a90>
<Element '{urn:mpeg:mpeg7:schema:2001}AudioDescriptor' at 0x7f54d3f84210>
